In [61]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [35]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [56]:
# 기본 코드

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import pymysql

model_path = "hair_segmenter.tflite"
# Height and width that will be used by the model
DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480
def resize_and_show(image):
    h, w = image.shape[:2]
    if h < w:
        img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
    else:
        img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
    cv2.imshow('Image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

salon_id = 'a000'
afterImg=''

sql = f"select pic_path from my_history where salon_id = '{salon_id}'"
conn = pymysql.connect(host='project-db-stu.smhrd.com',port=3307, user='campus_b_230519_3',password='smhrd3',db='campus_b_230519_3',charset='utf8')
cur = conn.cursor()
cur.execute(sql)
result = cur.fetchall()
afterImg = result[-1][0]

#with conn:
#     with conn.cursor() as cur:
#        cur.execute(sql,(salon_id))
#        result = cur.fetchall()
#        afterImg = result[-1][0]
#        print(afterImg)

# 이미지 불러올 폴더
IMAGE_FOLDER = 'C:/Users/user/git/final_project_t3/SmartMirror/src/main/resources/static/afterImg'

IMAGE_FILENAMES = [afterImg]

for name in IMAGE_FILENAMES:
    image_path = os.path.join(IMAGE_FOLDER, name)+'.jpg'

# Performs resizing and showing the image
image_path=image_path
images = {name: cv2.imread(image_path) for name in IMAGE_FILENAMES}
images.items()

BG_COLOR = (192, 192, 192) # gray
MASK_COLOR = (255, 255, 255) # white


# options
base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.ImageSegmenterOptions(base_options=base_options,
                                       output_category_mask=True)

# 세그먼트 샘플링
# segmenter 생성
with vision.ImageSegmenter.create_from_options(options) as segmenter:

  # 이미지폴더를 순회하며 이미지파일을 읽어옴
    for name in IMAGE_FILENAMES:
        image_file_path = os.path.join(IMAGE_FOLDER, name)+'.jpg'

      # Create the MediaPipe image file that will be segmented
        image = mp.Image.create_from_file(image_file_path)

      # Retrieve the masks for the segmented image
        segmentation_result = segmenter.segment(image)
        category_mask = segmentation_result.category_mask
      
      # Generate solid color images for showing the output segmentation mask.
        image_data = image.numpy_view()
        fg_image = np.zeros(image_data.shape, dtype=np.uint8)
        fg_image[:] = MASK_COLOR
        bg_image = np.zeros(image_data.shape, dtype=np.uint8)
        bg_image[:] = BG_COLOR

        condition = np.stack((category_mask.numpy_view(),) * 3, axis=-1) > 0.2
        output_image = np.where(condition, fg_image, bg_image)
    
# 헤어 세그먼트(머리카락만 추출)


with vision.ImageSegmenter.create_from_options(options) as segmenter:
       
  # Loop through demo image(s)
    for name in IMAGE_FILENAMES:
        image_file_path = os.path.join(IMAGE_FOLDER, name)

      # Create the MediaPipe image file that will be segmented
        image = mp.Image.create_from_file(image_file_path+'.jpg')

      # Retrieve the masks for the segmented image
        segmentation_result = segmenter.segment(image)
        category_mask = segmentation_result.category_mask
      
        image_data = cv2.cvtColor(image.numpy_view(), cv2.COLOR_BGR2RGB)
      # category_mask 를 넘파이 배열로 변환
        category_mask = category_mask.numpy_view()
      
      # Apply hair mask to the original image
        output_image = cv2.bitwise_and(image_data, image_data, mask=category_mask)
      
      # Create a mask for the black background
        black_bg_mask = np.all(output_image == [0, 0, 0], axis=2)

      # Create an alpha channel with the black background mask
        alpha_channel = np.where(black_bg_mask, 0, 255).astype(np.uint8)

      # Add the alpha channel to the hair image
        hair_image_with_alpha = cv2.cvtColor(output_image, cv2.COLOR_BGR2BGRA)
        hair_image_with_alpha[:, :, 3] = alpha_channel

      # 이미지 저장
        sql2 = "select sequence from hair_style_img"
        seq = ''

        cur.execute(sql2)
        result = cur.fetchall()
        seq = result[-1][0]+1
        seq = str(seq)
        print(seq)
        conn.close()
        
        output_path = 'C:/Users/user/git/final_project_t3/SmartMirror/src/main/resources/static/hairStyle/hairStyle_'+salon_id+'_'+seq+'.jpg'
        cv2.imwrite(output_path, hair_image_with_alpha)
    
      # Show the image with hair segmentation
        resize_and_show(hair_image_with_alpha)

13


In [59]:
# Flask 서버에서 구동
# 스프링 http://59.0.234.211:8087/pytest 과 Flask http://59.0.234.211:5000/testpage 연동

from flask import Flask

import urllib.request
import pymysql
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import requests

app = Flask(__name__)

@app.route("/testpage")
def test() :
    model_path = "hair_segmenter.tflite"
    # Height and width that will be used by the model
    DESIRED_HEIGHT = 480
    DESIRED_WIDTH = 480
    def resize_and_show(image):
        h, w = image.shape[:2]
        if h < w:
            img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
        else:
            img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
        cv2.imshow('Image', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    # 미용실 id 설정
    salon_id = 'a000'
    afterImg=''
    
    # db연결, sql문 작성
    conn = pymysql.connect(host='project-db-stu.smhrd.com',port=3307, user='campus_b_230519_3',password='smhrd3',db='campus_b_230519_3',charset='utf8')
    cur = conn.cursor()
    
    # 결과값의 마지막 값 선택
    sql = f"select pic_path from my_history where salon_id = '{salon_id}'"   
    cur.execute(sql)
    result = cur.fetchall()
    afterImg = result[-1][0]
    
    #with를 사용하면 바로 close 되어서 아래 쿼리 작동이 안 됨
    #with conn:
    #     with conn.cursor() as cur:
    #        cur.execute(sql,(salon_id))
    #        result = cur.fetchall()
    #        afterImg = result[-1][0]
    #        print(afterImg)

    # 이미지 불러올 폴더
    IMAGE_FOLDER = 'C:/Users/user/git/final_project_t3/SmartMirror/src/main/resources/static/afterImg'

    IMAGE_FILENAMES = [afterImg]

    for name in IMAGE_FILENAMES:
        image_path = os.path.join(IMAGE_FOLDER, name)+'.jpg'

    # Performs resizing and showing the image
    image_path=image_path
    images = {name: cv2.imread(image_path) for name in IMAGE_FILENAMES}
    images.items()

    BG_COLOR = (192, 192, 192) # gray
    MASK_COLOR = (255, 255, 255) # white

    # options
    base_options = python.BaseOptions(model_asset_path=model_path)
    options = vision.ImageSegmenterOptions(base_options=base_options,
                                           output_category_mask=True)

    # 세그먼트 샘플링
    # segmenter 생성
    with vision.ImageSegmenter.create_from_options(options) as segmenter:

      # 이미지폴더를 순회하며 이미지파일을 읽어옴
        for name in IMAGE_FILENAMES:
            image_file_path = os.path.join(IMAGE_FOLDER, name)+'.jpg'

          # Create the MediaPipe image file that will be segmented
            image = mp.Image.create_from_file(image_file_path)

          # Retrieve the masks for the segmented image
            segmentation_result = segmenter.segment(image)
            category_mask = segmentation_result.category_mask

          # Generate solid color images for showing the output segmentation mask.
            image_data = image.numpy_view()
            fg_image = np.zeros(image_data.shape, dtype=np.uint8)
            fg_image[:] = MASK_COLOR
            bg_image = np.zeros(image_data.shape, dtype=np.uint8)
            bg_image[:] = BG_COLOR

            condition = np.stack((category_mask.numpy_view(),) * 3, axis=-1) > 0.2
            output_image = np.where(condition, fg_image, bg_image)

    # 헤어 세그먼트(머리카락만 추출)


    with vision.ImageSegmenter.create_from_options(options) as segmenter:

      # Loop through demo image(s)
        for name in IMAGE_FILENAMES:
            image_file_path = os.path.join(IMAGE_FOLDER, name)

          # Create the MediaPipe image file that will be segmented
            image = mp.Image.create_from_file(image_file_path+'.jpg')

          # Retrieve the masks for the segmented image
            segmentation_result = segmenter.segment(image)
            category_mask = segmentation_result.category_mask

            image_data = cv2.cvtColor(image.numpy_view(), cv2.COLOR_BGR2RGB)
          # category_mask 를 넘파이 배열로 변환
            category_mask = category_mask.numpy_view()

          # Apply hair mask to the original image
            output_image = cv2.bitwise_and(image_data, image_data, mask=category_mask)

          # Create a mask for the black background
            black_bg_mask = np.all(output_image == [0, 0, 0], axis=2)

          # Create an alpha channel with the black background mask
            alpha_channel = np.where(black_bg_mask, 0, 255).astype(np.uint8)

          # Add the alpha channel to the hair image
            hair_image_with_alpha = cv2.cvtColor(output_image, cv2.COLOR_BGR2BGRA)
            hair_image_with_alpha[:, :, 3] = alpha_channel

          # 헤어 스타일 이미지 테이블 시퀀스 값 조회 후 가장 마지막 값 선택하여 다음 값을 파일 이름에 더해준다.
            sql2 = "select sequence from hair_style_img"
            seq = ''
   
            cur.execute(sql2)
            result = cur.fetchall()
            seq = result[-1][0]+1
            seq = str(seq)
            print(seq)
            
          # 헤어 스타일 이미지 파일명
            hair_id = 'hairStyle_'+salon_id+'_'+seq
          
          # 헤어 스타일 이미지 저장 경로
            output_path = 'C:/Users/user/git/final_project_t3/SmartMirror/src/main/resources/static/hairStyle/'+hair_id+'.jpg'
            cv2.imwrite(output_path, hair_image_with_alpha)
          
          # 헤어 스타일 이미지 DB 저장
            sql3 = f"insert into hair_style_img (hair_id, salon_id) values ('{hair_id}','{salon_id}')"
            cur.execute(sql3)
            conn.commit()
            
            conn.close()
          # Show the image with hair segmentation
          # resize_and_show(hair_image_with_alpha)

    return "YYYYY"

test()

#-------------------------------------------
# DB 저장 테스트용 코드
# @app.route("/testpage2")
# def spring():
#     member_id = 'bb'
#     salon_id = 'a000'

#     conn = pymysql.connect(host='project-db-stu.smhrd.com',port=3307, user='campus_b_230519_3',password='smhrd3',db='campus_b_230519_3',charset='utf8')

#     sql = f"insert into test (id, pw) values ('{member_id}','{salon_id}')"

#     cur = conn.cursor()

#     cur.execute(sql)

#     conn.commit()
    
#     return "DB에 저장"

# spring()

if __name__ == '__main__':
    app.run(debug=False,host="59.0.234.211",port=5000)

15
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://59.0.234.211:5000
Press CTRL+C to quit
